# classify

ref: https://github.com/PratyushTripathy/Landsat-Classification-Using-Convolution-Neural-Network/tree/master

https://github.com/weecology/DeepTreeAttention/blob/main/README.md (attention + pylighting fw)

## prepare labels

In [ ]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np

ref: 
https://github.com/klwalker-sb/burntfields_punjab

https://github.com/aime-valentin/tree_species_predictions/tree/master

https://github.com/swcoughlan/seaweed-classification

https://github.com/MitaliBhurani/Delineating-urban-areas-from-satellite-imagery/blob/master/Sentinel_imbalaced_moradabad_cv.ipynb

https://github.com/ML-MachineLearning/randomforest-GA/blob/master/random_forest.ipynb

https://github.com/AgataKisel/imagery_classification-/blob/main/random_forest.py

In [1]:
import rasterio
raster_path = r'D:\Sync\research\tree_species_estimation\tree_dataset\rmf\processed\20m\labels\tiles_128\tile_0_31.tif'
src  = rasterio.open(raster_path)
src.nodata

-1.0

In [ ]:
import os
import torch
import rasterio
from torch.utils.data import Dataset
import numpy as np

class TreeSpeciesDataset(Dataset):
    def __init__(self, tiles_dir, labels_dir, tile_names):
        """
        Args:
            tiles_dir (str): Directory containing the input tiles (12 bands).
            labels_dir (str): Directory containing the label tiles (9 bands).
            tile_names (list): List of tile filenames to load (e.g., 'tile_x_y.tif').
        """
        self.tiles_dir = tiles_dir
        self.labels_dir = labels_dir
        self.tile_names = tile_names

    def __len__(self):
        return len(self.tile_names)

    def __getitem__(self, idx):
        tile_name = self.tile_names[idx]

        # Load the input tile (12 bands)
        input_tile_path = os.path.join(self.tiles_dir, tile_name)
        with rasterio.open(input_tile_path) as src:
            input_data = src.read()  # Shape: (12, H, W)
            nodata_value_input = src.nodata  # NoData value for the input

        # Load the target tile (9 bands)
        label_tile_path = os.path.join(self.labels_dir, tile_name)
        with rasterio.open(label_tile_path) as src:
            target_data = src.read()  # Shape: (9, H, W)
            nodata_value_target = src.nodata  # NoData value for the target

        # Create masks for NoData pixels (True where NoData)
        input_mask = input_data == nodata_value_input
        target_mask = target_data == nodata_value_target

        # Convert NoData to -1 in both input and target for easier handling
        input_data = np.where(input_mask,-1, input_data)
        target_data = np.where(target_mask, -1, target_data)

        # Create a combined mask (where either input or target has NoData)
        combined_mask = np.any(input_mask, axis=0) | np.any(target_mask, axis=0)

        # Convert data to PyTorch tensors
        input_tensor = torch.from_numpy(input_data).float()
        target_tensor = torch.from_numpy(target_data).float()
        mask_tensor = torch.from_numpy(combined_mask).bool()  # Convert mask to PyTorch boolean tensor

        return input_tensor, target_tensor, mask_tensor

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=12, out_channels=9):
        super(UNet, self).__init__()

        # Encoder
        self.enc_conv0 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.enc_conv1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.enc_conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        # Decoder
        self.dec_conv3 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec_conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec_conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.dec_conv0 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

        # Maxpool and upsample
        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.enc_conv0(x))
        x2 = self.pool(x1)
        x2 = F.relu(self.enc_conv1(x2))
        x3 = self.pool(x2)
        x3 = F.relu(self.enc_conv2(x3))
        x4 = self.pool(x3)
        x4 = F.relu(self.enc_conv3(x4))

        # Decoder
        x = self.up(x4)
        x = F.relu(self.dec_conv3(x))
        x = self.up(x)
        x = F.relu(self.dec_conv2(x))
        x = self.up(x)
        x = F.relu(self.dec_conv1(x))
        x = F.relu(self.dec_conv0(x))  # Output layer without activation

        return x


In [ ]:
from torch.utils.data import DataLoader

def load_tile_names(file_path):
    """
    Load tile names from a .txt file.

    Args:
        file_path (str): Path to the .txt file.

    Returns:
        tile_names (list): List of tile names.
    """
    with open(file_path, 'r') as f:
        tile_names = f.read().splitlines()
    return tile_names

# Directories
tiles_dir = "s2/tiles_128"  # Directory containing the 12-band Sentinel imagery
labels_dir = "labels/tiles_128"  # Directory containing the 9-band species proportions

# Load tile names for each split
train_tile_names = load_tile_names("train.txt")
val_tile_names = load_tile_names("validation.txt")
test_tile_names = load_tile_names("test.txt")

# Create datasets
train_dataset = TreeSpeciesDataset(tiles_dir, labels_dir, train_tile_names)
val_dataset = TreeSpeciesDataset(tiles_dir, labels_dir, val_tile_names)
test_dataset = TreeSpeciesDataset(tiles_dir, labels_dir, test_tile_names)

# Create DataLoaders
batch_size = 4  # Adjust based on your memory capacity
num_workers = 4  # Adjust based on your system

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = UNet(in_channels=12, out_channels=9)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training parameters
num_epochs = 10  # Adjust as needed

# Training loop
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets, mask in train_loader:
        inputs = inputs.to(device)  # Shape: (batch_size, 12, H, W)
        targets = targets.to(device)  # Shape: (batch_size, 9, H, W)
        mask = mask.to(device)  # Shape: (batch_size, H, W)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)  # Shape: (batch_size, 9, H, W)

        # Apply the mask to ignore NoData pixels in the loss calculation
        valid_outputs = outputs[:, :, ~mask]
        valid_targets = targets[:, :, ~mask]

        # Compute loss only for valid pixels
        loss = criterion(valid_outputs, valid_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}")

    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets, mask in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            mask = mask.to(device)

            outputs = model(inputs)

            # Apply the mask to ignore NoData pixels
            valid_outputs = outputs[:, :, ~mask]
            valid_targets = targets[:, :, ~mask]

            loss = criterion(valid_outputs, valid_targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}")

In [ ]:
# Save the trained model
model_save_path = "tree_species_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Testing step
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")


pylighting

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F

class UNetLightning(pl.LightningModule):
    def __init__(self, in_channels=12, out_channels=9, learning_rate=1e-3):
        super(UNetLightning, self).__init__()

        # Define the U-Net architecture
        self.enc_conv0 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.enc_conv1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.enc_conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        self.dec_conv3 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.dec_conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.dec_conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.dec_conv0 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.learning_rate = learning_rate
        self.criterion = nn.MSELoss()  # You can also experiment with other losses if needed

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.enc_conv0(x))
        x2 = self.pool(x1)
        x2 = F.relu(self.enc_conv1(x2))
        x3 = self.pool(x2)
        x3 = F.relu(self.enc_conv2(x3))
        x4 = self.pool(x3)
        x4 = F.relu(self.enc_conv3(x4))

        # Decoder
        x = self.up(x4)
        x = F.relu(self.dec_conv3(x))
        x = self.up(x)
        x = F.relu(self.dec_conv2(x))
        x = self.up(x)
        x = F.relu(self.dec_conv1(x))
        x = self.dec_conv0(x)

        return x

    def training_step(self, batch, batch_idx):
        inputs, targets, mask = batch
        outputs = self(inputs)

        # Apply the mask to exclude NoData pixels
        valid_outputs = outputs[:, :, ~mask]
        valid_targets = targets[:, :, ~mask]

        # Compute the loss only for valid pixels
        loss = self.criterion(valid_outputs, valid_targets)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets, mask = batch
        outputs = self(inputs)

        # Apply the mask to exclude NoData pixels
        valid_outputs = outputs[:, :, ~mask]
        valid_targets = targets[:, :, ~mask]

        loss = self.criterion(valid_outputs, valid_targets)
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class TreeSpeciesDataModule(pl.LightningDataModule):
    def __init__(self, tiles_dir, labels_dir, train_tile_names, val_tile_names, test_tile_names, batch_size=4, num_workers=4):
        super().__init__()
        self.tiles_dir = tiles_dir
        self.labels_dir = labels_dir
        self.train_tile_names = train_tile_names
        self.val_tile_names = val_tile_names
        self.test_tile_names = test_tile_names
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        # Create datasets for train, val, test splits
        self.train_dataset = TreeSpeciesDataset(self.tiles_dir, self.labels_dir, self.train_tile_names)
        self.val_dataset = TreeSpeciesDataset(self.tiles_dir, self.labels_dir, self.val_tile_names)
        self.test_dataset = TreeSpeciesDataset(self.tiles_dir, self.labels_dir, self.test_tile_names)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Initialize the data module
data_module = TreeSpeciesDataModule(
    tiles_dir="s2/tiles_128",
    labels_dir="labels/tiles_128",
    train_tile_names=load_tile_names("train.txt"),
    val_tile_names=load_tile_names("validation.txt"),
    test_tile_names=load_tile_names("test.txt"),
    batch_size=4,
    num_workers=4
)

# Initialize the model
model = UNetLightning(in_channels=12, out_channels=9, learning_rate=1e-3)

# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Track the validation loss
    filename='best-model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,  # Only save the best model
    mode='min'  # We want to minimize the validation loss
)

# Create a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=10,
    gpus=1 if torch.cuda.is_available() else 0,  # Use GPU if available
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, data_module)

# Test the model after training
trainer.test(model, data_module)

# Save the best model after training
trainer.save_checkpoint("final_model.ckpt")
# Load the saved model
#model = UNetLightning.load_from_checkpoint("final_model.ckpt")

rf

ref: https://github.com/shelleygoel/sentinel2-land-cover-classifier/tree/main

- Key Points:
  - X (Features): Sentinel imagery tiles stored in s2/tiles_128/ (each tile has 12 bands, size 128x128).
  - Y (Labels): The species composition tiles stored in labels/tiles_128/ (each tile has 9 bands, size 128x128). The target for each pixel is a 9-element vector representing species proportions.
  - Train/Validation/Test Splits: The tiles to use for training, validation, and testing are specified in train.txt, validation.txt, and test.txt.
- Step-by-Step Implementation:
  - Loading Data: We'll read all 1060 tiles from the directories for both input (X) and target (Y).
  - Random Forest: We'll use RandomForestRegressor to fit the data.
  - Training/Validation/Test Splits: These splits are defined by the .txt files.
  - Pixel-Wise Classification: The model will predict the species proportions for each pixel.

In [ ]:
import os
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Function to load tiles (X) and labels (Y)
def load_tile_data(tile_names, tiles_dir, labels_dir):
    """
    Load the imagery (X) and label (Y) data for the given tile names.

    Args:
        tile_names (list): List of tile names to load.
        tiles_dir (str): Directory containing the Sentinel imagery (X).
        labels_dir (str): Directory containing the species composition labels (Y).

    Returns:
        X (np.array): Flattened feature array (pixels x 12).
        Y (np.array): Flattened label array (pixels x 9).
    """
    X_list, Y_list = [], []
    print("loading data...")
    for tile_name in tqdm(tile_names):
        # Define paths for the input and label tiles
        input_tile_path = os.path.join(tiles_dir, tile_name)
        label_tile_path = os.path.join(labels_dir, tile_name)
        
        # Load input (12 bands) and label (9 bands) tiles
        with rasterio.open(input_tile_path) as src_x:
            X = src_x.read()  # Shape: (12, 128, 128)

        with rasterio.open(label_tile_path) as src_y:
            Y = src_y.read()  # Shape: (9, 128, 128)

        # Reshape to (num_pixels, num_bands)
        X_flat = X.reshape(X.shape[0], -1).T  # Shape: (num_pixels, 12)
        Y_flat = Y.reshape(Y.shape[0], -1).T  # Shape: (num_pixels, 9)

        # Append to lists
        X_list.append(X_flat)
        Y_list.append(Y_flat)
    
    # Concatenate all tiles into a single array
    X_all = np.vstack(X_list)
    print(f"shape of labels: {X_all.shape}")
    Y_all = np.vstack(Y_list)
    print(f"shape of labels: {Y_all.shape}")
    
    return X_all, Y_all

# Function to read the train/validation/test splits
def load_split(file_path):
    """
    Load the tile names from the train/validation/test split files.

    Args:
        file_path (str): Path to the split .txt file.

    Returns:
        tile_names (list): List of tile names in the split.
    """
    with open(file_path, 'r') as file:
        tile_names = file.read().splitlines()
    return tile_names

# Set up directories
directory = r"D:\Sync\research\tree_species_estimation\tree_dataset\rmf\processed\10m"
tiles_dir = os.path.join(directory, "rmf_s2", "summer", "tiles_128")  # Directory for X
labels_dir = os.path.join(directory, "labels", "tiles_128")  # Directory for Y

# Load train/validation/test splits
train_tile_names = load_split(os.path.join(directory, "dataset", "train_tiles.txt"))[:400]
val_tile_names = load_split(os.path.join(directory, "dataset", "val_tiles.txt"))[:100]

# Load the training data
X_train, Y_train = load_tile_data(train_tile_names, tiles_dir, labels_dir)

# Load the validation data (optional, but useful for hyperparameter tuning)
X_val, Y_val = load_tile_data(val_tile_names, tiles_dir, labels_dir)

# Initialize and train the Random Forest model
print("start training...")
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, Y_train)

# Evaluate on the validation set
print("validating...")
Y_val_pred = rf.predict(X_val)
val_mse = mean_squared_error(Y_val, Y_val_pred)
print(f"Validation Mean Squared Error: {val_mse}")


loading data...


100%|██████████| 400/400 [01:40<00:00,  3.99it/s]


shape of labels: (6553600, 12)
shape of labels: (6553600, 9)
loading data...


100%|██████████| 100/100 [00:34<00:00,  2.93it/s]


shape of labels: (1638400, 12)
shape of labels: (1638400, 9)
start training...
validating...
Validation Mean Squared Error: 0.21201176008304376


In [ ]:
import joblib
# After training the model
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, Y_train)

# Save the model to a file
model_filename = 'random_forest_model.joblib'
joblib.dump(rf, model_filename)

print(f"Model saved as {model_filename}")

# Load the saved model
loaded_rf = joblib.load(model_filename)

# Load the testing data (for final evaluation)
test_tile_names = load_split(os.path.join(directory, "dataset", "test_tiles.txt"))
X_test, Y_test = load_tile_data(test_tile_names, tiles_dir, labels_dir)

# Use the loaded model to make predictions
Y_test_pred = loaded_rf.predict(X_test)
test_mse = mean_squared_error(Y_test, Y_test_pred)
print(f"Test Mean Squared Error with loaded model: {test_mse}")


LUCinSA_helpers
Helper functions and notebooks to interact with data on High-Performance Computing environment, designed to be used in conjunction with processing guide for remote sensing projects on Land-Use Change in Latin America:

https://github.com/klwalker-sb/LUCinSA_helpers/tree/master

https://klwalker-sb.github.io/LUCinLA_stac/Downloading.html